Será que podemos comparar ideologias políticas a partir de tweets?

ideologia: conjunto de convicções filosóficas, sociais, políticas etc. de um indivíduo ou grupo de indivíduos.

Comparações:
    - análise gráfica comparativa da frequência de palavras ou relacionadas a tópicos: 
        religião, educação, orientação sexual, identidade de gênero, economia, saúde, porte de armas, combate a pobreza
            ex: Quantidade de tweets do Bolsonaro falando sobre drogas é muito maior que a do Lula, que mostra maior favor a política anti-drogas
            (Buscar se a quantidade de palavras segue a mesma tendência)

    -! análise comparativa de tweets antes e depois de eleições

Observações:
    - contam retweets? -> Usar a função clean() para filtrar

Proximos passos:
Otimizar função de limpeza -> lemanização e estemização para remoção de radicais
Comparar temas por datas específicas

In [ ]:
# Imports section
import json
import nltk
import pandas as pd

In [ ]:
def get_data():
    '''Open and read dataset files, saving contents as raw data.'''
    with open('dataset/jairbolsonaro.json','r') as file_01:
        data_01=json.load(file_01)
    with open('dataset/LulaOficial.json','r') as file_02:
        data_02=json.load(file_02)
        
    return data_01,data_02
# get_data()[0][0]

In [ ]:
def extract_tweets():
    '''Store tweets in lists without metadata'''
    data_01,data_02 = get_data()
    bolso_tweets = []
    for el in data_01:
        bolso_tweets.append(el['full_text'])

    lula_tweets = []
    for el in data_02:
        lula_tweets.append(el['full_text'])
        
    return bolso_tweets, lula_tweets

In [ ]:
def compare(bolso_tweets,lula_tweets,subject):
    # Fix_01:Currently isn't useful because it is too restricted, doesn't account for synonyms and related words
     
    '''Compare tweets from both sides based on a subject'''
    # Look into Bolsonaro's tweets for the subject and store tweets in a list
    b_selected = []
    for tweet in bolso_tweets:
        if subject.lower() in tweet.lower():
            b_selected.append(tweet)

    # Look into Lula's tweets for the subject and store tweets in a list
    l_selected = []
    for tweet in lula_tweets:
        if subject.lower() in tweet.lower():
            l_selected.append(tweet)

    # Create two series based on the selected tweets lists
    b = pd.Series(b_selected,name='Bolsonaro')
    l = pd.Series(l_selected,name='Lula')

    # Create dataframe concatenating the two series
    pd.set_option('display.max_colwidth', None)
    df=pd.concat([b,l],axis=1)
    
    return df

In [ ]:
def clean(tweet):
    # Limited amount of stopwords, needs to remove links, emogis, replies, numbers
    '''Remove stopwords from tweet'''
    # nltk.download('stopwords')
    stopwords = nltk.corpus.stopwords.words('portuguese')
    sw_extended=['ser', 'ter', 'têm', 'havia']
    stopwords.append(sw_extended)
    for word in tweet: 
        if word in stopwords:
            tweet.remove(word)
    return tweet

In [ ]:
def count_words(tweets,min=0):
    '''Count the ocurrence of key words in the tweets'''
    words={}
    selected_words=[]
    for tweet in tweets:
        for word in clean(tweet.split()):
            if word in words.keys():
                words[word] += 1
            else:
                words[word] = 1

                
    # Filter the words into a list based on a minimum number of ocurrences
    for word, ocurrence in words.items():
        if ocurrence > min:
            selected_words.append(word)

    # return selected_words
    return sorted(words,key=words.get,reverse=True)

In [ ]:
bolso_tweets, lula_tweets = extract_tweets()
count_words(bolso_tweets)